In [2]:
using JuMP, GLPK

In [3]:
model = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK